## Wczytywanie danych, zapisywanie danych i walidacja danych w PySpark

source: Layla AI, *PySpark Essentials for Data Scientists (Big Data + Python)*

## Wczytywanie danych

### Dane csv
**Source:** https://www.kaggle.com/spscientist/students-performance-in-exams

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
# Może chwilę potrwać
spark = SparkSession.builder.appName("UczymySięSparka").getOrCreate()
spark

# Zaczynamy od wczytania csvki
# Dwa parametry, na które trzeba wzrócić uwagę: inferSchema & header

# Some csv data
students = spark.read.csv('StudentsPerformance.csv',inferSchema=True,header=True) #zwrócić uwagę na inferSchema i header!

### Pliki parquet ###

Parquet to najbardziej popularny format danych w Big Data. Powód? Są bardziej skompresowane nawet niż pliki .zip.

In [2]:
parquet = spark.read.parquet('users1.parquet')
parquet.show(2)

+-------------------+---+----------+---------+----------------+------+--------------+----------------+---------+---------+---------+----------------+--------+
|  registration_dttm| id|first_name|last_name|           email|gender|    ip_address|              cc|  country|birthdate|   salary|           title|comments|
+-------------------+---+----------+---------+----------------+------+--------------+----------------+---------+---------+---------+----------------+--------+
|2016-02-03 08:55:29|  1|    Amanda|   Jordan|ajordan0@com.com|Female|   1.197.201.2|6759521864920116|Indonesia| 3/8/1971| 49756.53|Internal Auditor|   1E+02|
|2016-02-03 18:04:03|  2|    Albert|  Freeman| afreeman1@is.gd|  Male|218.111.175.34|                |   Canada|1/16/1968|150280.17|   Accountant IV|        |
+-------------------+---+----------+---------+----------------+------+--------------+----------------+---------+---------+---------+----------------+--------+
only showing top 2 rows



### Wiele plików parquet ### 

Większość dużych zbiorów danych będzie podzielona na mniejsze części. Używając *, można je wczytać jednym poleceniem.

In [4]:
partitioned = spark.read.parquet('users*') #asteriks
partitioned.show(2)

Można też zdecydować się na wczytanie tylko kilku plików ze zbioru: 

In [ ]:
users1_2 = spark.read.option("basePath").parquet('users1.parquet', 'users2.parquet') #path jest odwołaniem co current working directory
users1_2.show()

### Walidacja danych

Poniżej przykłady jak sprawdzić czy dane zostały załadowane poprawnie. Zaczynamy od podstaw (poprawne typy zmiennych, poprawnie załadowane wartości etc.), do walidacji pod modelowanie jeszcze dotrzemy.

In [ ]:
# Podgląd ramki - sparkowy
students.show(3)

In [ ]:
# Podgląd ramki poprzez wykorzystanie pandas API
students.limit(5).toPandas()

In [ ]:
# Upewnijmy się, że to są dwa różne rodzaje danych:
print(type(students))
studentsPdf = students.toPandas()
print(type(studentsPdf))

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


Ogólne, skondensowane informacje o ramce:

In [ ]:
# podobne do df.head()
print(students.printSchema())
print("")
print(students.columns)
print("")
print(students.describe()) 

Podgląd typu jednej kolumny:

In [ ]:
students.schema['math score'].dataType

Ładniejszy wynik describe:

In [ ]:
students.describe(['math score']).show()

Wywoływanie funkcji summary:

In [ ]:
students.select("math score", "reading score","writing score").summary("count", "min", "25%", "75%", "max").show()
# Tu podglądaliśmy kwartyle, ale można też podejrzeć średnią, mediane etc.

### Specyfikacja typu wczytywanych danych

Niektóre pliki łatwiej załadować prawidłowo niż inne (tj. wykorzystać istniejący schemat, fachowo: odziedziczyć) - np. dane w formie tabelarycznej takie jak pliki csv. Niestety nierzadko trzeba samodzielnie określić kształt ramki wynikowej, zwłaszcza jeżeli korzysta się z metody, która nie używa parametru .inferSchema(). 
Poniżej przykład jak sobie z tym poradzić. Trzeba zacząć od zaimportowania dodatkowych modułów. 

In [19]:
from pyspark.sql.types import StructField,StringType,IntegerType,StructType,DateType

Następnie, korzystając z poniższego klucza, uzupełniamy parametry:
 * param name: string - nazwa pola.
 * param dataType: :class:`DataType` - typ pola.
 * param nullable: - boolean, czy może przyjmować wartości null.

Wartości dla powyższych parametrów zapisujemy jako listę, którą potem przekażemy jako argument do metody wczytującej dane:

In [20]:
data_schema = [StructField("name", StringType(), True),
               StructField("email", StringType(), True),
               StructField("city", StringType(), True),
               StructField("mac", StringType(), True),
               StructField("timestamp", DateType(), True),
               StructField("creditcard", StringType(), True)]

In [21]:
final_struc = StructType(fields=data_schema)

Sprawdźmy jak to działa w praktyce. Tym razem wczytamy .jsona 

Plik można znaleźć tu: https://gist.github.com/raine/da15845f332a2fb8937b344504abfbe0

In [22]:
people = spark.read.json('people.json', schema=final_struc)

Sprawdzamy Schemę:

In [23]:
people.printSchema()

root
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- city: string (nullable = true)
 |-- mac: string (nullable = true)
 |-- timestamp: date (nullable = true)
 |-- creditcard: string (nullable = true)



## Export danych

Najpierw prosta csv-ka. Składnia generalnie:

In [19]:

df.write.csv('foo.csv', header=True)

Można też wykorzystać API pandasowe:

In [ ]:
students.toPandas().to_csv('foo.csv')

Plik parquet:

In [ ]:
df.write.parquet('bar.parquet')

Pliki parquet podzielone na partycje:

In [ ]:
users1_2.write.partitionBy("gender").parquet('part_parquet/')

Można też stworzyć własną ramkę danych i ją eksportować:

In [ ]:
values = [('Pear',10),('Orange',36),('Banana',123),('Kiwi',48),('Peach',16),('Strawberry',1)]
df = spark.createDataFrame(values,['fruit','quantity'])
df.show()

# Agregacja danych w PySpark:

 Podstawy ramek mamy za sobą, teraz pora na agregację danych. Polecenia, które przedstawiamy poniżej, są bardzo potrzebne do porządnego sprawdzenia jakości danych. Przydadzą się także do odpowiedzi na proste pytania biznesowe.

Przed nami polecenia/zagadnienia:

 - GroupBy
 - Pivot
 - Metody agregacyjne
 - Kombo trzech powyższych

### Wczytujemy dane do zadań:

Poniższy plik zawiera dane o ogłoszeniach z Air BNB z Nowego Jorku z 2019. Jedna linijka = jeden booking.

Żródło danych: https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data/data

Ruszamy. Najpierw obejrzymy sobie te dane:

In [ ]:
# cska:
airbnb = spark.read.csv('nyc_air_bnb.csv',inferSchema=True,header=True) # parametry: inferSchema i header (!)

In [3]:
airbnb.limit(5).toPandas()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.9419,Private room,150,3,0,None,None,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [4]:
print(airbnb.printSchema())

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- minimum_nights: string (nullable = true)
 |-- number_of_reviews: string (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: string (nullable = true)
 |-- calculated_host_listings_count: string (nullable = true)
 |-- availability_365: integer (nullable = true)

None


Niektóre z kolumn mają zaczytany nieprawidłowy typ danych. Trzeba to poprawić używając funkcji .withColumn(). Jest to funkcja, którą można wykonać zmianę wartości kolumny, zmienić typ kolumny, ale także stworzyć nową kolumnę.

In [5]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

df = airbnb.withColumn("price", airbnb["price"].cast(IntegerType())) \
        .withColumn("minimum_nights", airbnb["minimum_nights"].cast(IntegerType())) \
        .withColumn("number_of_reviews", airbnb["number_of_reviews"].cast(IntegerType())) \
        .withColumn("reviews_per_month", airbnb["reviews_per_month"].cast(IntegerType())) \
        .withColumn("calculated_host_listings_count", airbnb["calculated_host_listings_count"].cast(IntegerType()))
#QA
print(df.printSchema())
df.limit(5).toPandas()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: integer (nullable = true)
 |-- calculated_host_listings_count: integer (nullable = true)
 |-- availability_365: integer (nullable = true)

None


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.0,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.0,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.9419,Private room,150,3,0,None,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.0,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.0,1,0


### Grupowanie i agregowanie zmiennych

Grupowanie wykonuje się funkcją groupBy(). Agregowanie funkcją aggregate(). Często są one wykorzystywane w tym samym momencie. Nie bez znaczenia jest to, że żeby wykonać agregację najpierw należy przekazać Sparkowi informację po czym zbiór pogrupować. 

GroupBy() grupuje wiersze z tą samą wartością wybranej kolumny, np. sprzedaż w ujęciu miesięcznym. Po wykonaniu grupowania, można przejść do agregacji, która zamiast kilku wierszy zwraca ich sumę, średnią etc. 

Aggregate() można też używać bez groupBy(). Robi się to po to żeby obejrzeć statystyki zbioru danych, na którym się aktualnie pracuje. Sprawdźmy najpierw ile było ogłoszeń w rozbciu na dzielnice?

In [ ]:

# GroupBy() + count() (ale można by też: sum, min, max)
df.groupBy("neighbourhood_group").count().show(7)

+-------------------+-----+
|neighbourhood_group|count|
+-------------------+-----+
|         Douglaston|    1|
|             Queens| 5630|
|              Nadia|    1|
|            Midtown|    4|
|    Jackson Heights|    2|
|     Hell's Kitchen|    7|
|  Greenwich Village|    2|
+-------------------+-----+
only showing top 7 rows



Dorzućmy jakąś agregację: 


In [ ]:
# Przykład:
df.groupBy("neighbourhood_group").mean("price").show(5)

+-------------------+-----------------+
|neighbourhood_group|       avg(price)|
+-------------------+-----------------+
|         Douglaston|              1.0|
|             Queens|99.57690941385435|
|              Nadia|             null|
|            Midtown|              9.0|
|    Jackson Heights|             16.0|
+-------------------+-----------------+
only showing top 5 rows



Niby można też tak, ale trzeba się wtedy ograniczyć do jednej zmiennej w danej operacji, dlatego nie jest to preferowana przez nas metoda:

In [ ]:
df.groupBy("neighbourhood").agg({'price':'mean'}).show(5)

+-------------+----------+
|neighbourhood|avg(price)|
+-------------+----------+
|       Corona| 59.171875|
| Richmondtown|      78.0|
| Prince's Bay|     409.5|
|  Westerleigh|      71.5|
|   Mill Basin|    179.75|
+-------------+----------+
only showing top 5 rows



To jest najlepsza z zaproponych metod, można agregować więcej kolumn w jednej operacji:

In [ ]:
from pyspark.sql.functions import *
df.groupBy("neighbourhood").agg(min(df.price).alias("Min Price"),max(df.price).alias("Max Price")).show(5)

+-------------+---------+---------+
|neighbourhood|Min Price|Max Price|
+-------------+---------+---------+
|       Corona|       23|      359|
| Richmondtown|       78|       78|
| Prince's Bay|       85|     1250|
|  Westerleigh|       40|      103|
|   Mill Basin|       85|      299|
+-------------+---------+---------+
only showing top 5 rows



To też przyzwoity sposób:

In [ ]:

summary = df.summary("count", "min", "25%", "75%", "max")
summary.toPandas()
# UWAGA: do wykonania polecenia wykorzystywany jest CAŁY ZBIÓR -> memorry error.

,summary,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,count,49079,49047,48894,48873,48894,48894,48894,48894,48894,48887,48891,48738,38845,38858,48891,48737
1,min,"12 mins Manhattan""",1 Bed Apt in Utopic Williamsburg,"Heart of Greenwich Village""","very clean studio app""",194716858,2,-73.72247,-73.71299,-73.90783,-74,0,0,-73.94134,0,0,0
2,25%,9470528.0,2.4544724E7,7797690.0,475.0,1.94716858E8,40.68771,40.68981,-73.98309,56.0,69,1,1,0.76,0,1,0
3,75%,2.9151631E7,1.74786681E8,1.07434423E8,3.3499491E7,1.97400421E8,40.78304,40.76299,-73.93638,145.0,175,5,23,3.24,2,2,226
4,max,"獨一無二的紐約閣樓""","ﾏﾝﾊｯﾀﾝ､駅から徒歩4分でどこに行くのにも便利な場所!女性の方希望,ｷﾚｲなお部屋｡",呈刚,현선,Woodside,Woodside,West Village,Shared room,Shared room,10000,1250,629,9.66,58,365,365


Wynik powyżej jest nieczytelny, dużo lepszym sposobem byłoby wybranie kilku kolumn, które nas interesują:

In [11]:

# limit_summary = df.select("price","minimum_nights","number_of_reviews","last_review","reviews_per_month","calculated_host_listings_count","availability_365").summary("count","min","max")
limit_summary = df.select("price","minimum_nights","number_of_reviews").summary("count","min","max")
limit_summary.toPandas()

,summary,price,minimum_nights,number_of_reviews
0,count,48887,48891,48738
1,min,-74,0,0
2,max,10000,1250,629


### Agragacja ramki bez wykonywania groupBy: 
Wracamy do agregacji bez wykorzystania funkcji groupBy(). Robi się to używając funckji z biblioteki pyspark.sql:

In [25]:
# agregacja bez wykorzystania grupowania (krótsza forma df.groupBy.agg()).
# dostępne funkcje: min, max, count, countDistinct, approx_count_distinct
# df.agg.(covar_pop(col1, col2)) Nowa kolumna z wartością kowariancji kolumn col1 i col2
# df.agg.(covar_samp(col1, col2)) Nowa kolumna z wartością kowariancji próbki kolumn col1 i col2
# df.agg(corr(col1, col2)) Nowa koolumna z wartością korelacji Pearsona kolumn col1 i col2.

from pyspark.sql.functions import *
df.agg(min(df.price).alias("Min Price"),max(df.price).alias("Max Price")).show()

+---------+---------+
|Min Price|Max Price|
+---------+---------+
|      -74|    10000|
+---------+---------+



In [ ]:
# Można też tak i są to dość podobne do siebie metody
df.select(countDistinct("neighbourhood_group").alias('CountD'),avg('price'),stddev("price")).show()

In [ ]:
# Kolejna alternatywa, ale ta sama wada co w module wyżej - przyjmuje tylko jadną zmienną, więc nie jest to polecana metoda
df.agg({'number_of_reviews':'max'}).withColumnRenamed("max(number_of_reviews)", "Max Reviews").show()

### Funkcja pivot

Pivot() służy do wykonania tabeli krzyżowej. Trzeba jej używać w połączeniu z funkcją groupBy().

In [35]:
# Pivot Function
# pivot(pivot_col, values=None)
df.groupBy("room_type").pivot("neighbourhood_group", ["Queens", "Brooklyn"]).count().show(10)

+-----------+------+--------+
|  room_type|Queens|Brooklyn|
+-----------+------+--------+
|         51|  null|    null|
|        205|  null|    null|
|         54|  null|    null|
|        200|  null|    null|
|        279|  null|    null|
|        138|  null|    null|
|         69|  null|    null|
|         42|  null|    null|
|Shared room|   198|     413|
|  -73.95777|  null|    null|
+-----------+------+--------+
only showing top 10 rows



In [ ]:
# Wynik, w razie potrzeby, można też przefiltrować
# Powyżej uzyskaliśmy niewiele mówiący wynik
# Spróbujmy wybrać tylko "Shared room"
df.filter("room_type='Shared room'").groupBy("room_type").pivot("neighbourhood_group", ["Queens", "Brooklyn"]).count().show(100)

### Przykład połączenia wszystkich trzech:

Możliwe jest także połączenie wszystkich trzech metod w jednym zapytaniu: GroupBy, Pivot and Agg. Robi się to w ten sposób:

In [44]:
# from pyspark.sql.functions import *
df.groupBy("neighbourhood").pivot("neighbourhood_group", ["Queens", "Brooklyn"]).agg(min(df.price).alias("Min Price"),max(df.price).alias("Max Price")).toPandas()#.show()
# UWAGA: toPandas() używamy tylko jeżeli spodziewamy się otrzymać relatywnie mały zbiór wynikowy
# Wszystkie dane ładowane są do pamięci drivera:

,neighbourhood,Queens_Min Price,Queens_Max Price,Brooklyn_Min Price,Brooklyn_Max Price
0,Corona,23.0,359.0,NaN,NaN
1,Prince's Bay,NaN,NaN,NaN,NaN
2,Richmondtown,NaN,NaN,NaN,NaN
3,Mill Basin,NaN,NaN,85.0,299.0
4,Westerleigh,NaN,NaN,NaN,NaN
...,...,...,...,...,...
378,40.69383,NaN,NaN,NaN,NaN
379,Morningside Heights,NaN,NaN,NaN,NaN
380,Greenpoint,NaN,NaN,0.0,10000.0
381,Elmhurst,15.0,443.0,NaN,NaN
